In [1]:
import logging
logging.getLogger("py4j").setLevel(logging.INFO)
logging.getLogger("org.apache.spark").setLevel(logging.INFO)

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark with HDFS and Hive") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.warehouse.dir", "hdfs://namenode:9000/user/hive/warehouse") \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .config("spark.executor.memory", "2g") \
    .config("spark.executor.cores", "1") \
    .config("spark.driver.memory", "1g") \
    .enableHiveSupport() \
    .getOrCreate()

In [3]:
type(spark)

pyspark.sql.session.SparkSession

In [4]:
spark

Py4JJavaError: An error occurred while calling o41.conf.
: java.lang.IllegalStateException: LiveListenerBus is stopped.
	at org.apache.spark.scheduler.LiveListenerBus.addToQueue(LiveListenerBus.scala:92)
	at org.apache.spark.scheduler.LiveListenerBus.addToStatusQueue(LiveListenerBus.scala:75)
	at org.apache.spark.sql.internal.SharedState.<init>(SharedState.scala:115)
	at org.apache.spark.sql.SparkSession.$anonfun$sharedState$1(SparkSession.scala:143)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.SparkSession.sharedState$lzycompute(SparkSession.scala:143)
	at org.apache.spark.sql.SparkSession.sharedState(SparkSession.scala:142)
	at org.apache.spark.sql.SparkSession.$anonfun$sessionState$2(SparkSession.scala:162)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.SparkSession.sessionState$lzycompute(SparkSession.scala:160)
	at org.apache.spark.sql.SparkSession.sessionState(SparkSession.scala:157)
	at org.apache.spark.sql.SparkSession.conf$lzycom

In [ ]:
spark = SparkSession.builder \
    .config("master", "yarn") \
    .appName("new_spark") \
    .getOrCreate()

In [ ]:
spark.stop()

# Введение

In [ ]:
data = [1, 2, 3, 4, 5]
rdd = spark.sparkContext.parallelize(data)

In [ ]:
rdd

In [ ]:
print(rdd)

In [ ]:
rdd.collect()

In [ ]:
rdd.repartition(2)

In [ ]:
rdd

In [ ]:
rdd.repartition(2).collect()

In [ ]:
rdd_mapped = rdd.map(lambda x: x * 2)

In [ ]:
rdd_mapped

In [ ]:
rdd_mapped.collect()

In [ ]:
rdd_filtered = rdd.filter(lambda x: x % 2 == 0)

In [ ]:
rdd_filtered.collect()

In [ ]:
grouped = rdd.groupBy(lambda x: "even" if x % 2 == 0 else "odd")

In [ ]:
grouped.collect()

In [ ]:
print({k: list(v) for k, v in grouped.collect()})

# Экшены

In [ ]:
data = rdd.collect()

In [ ]:
data

In [ ]:
first_2 = rdd.take(3)

In [ ]:
first_2

In [ ]:
rdd.count()

In [ ]:
sum = rdd.reduce(lambda a, b: a + b)

In [ ]:
sum

### Передаем данные на драйвер: 

- collect()
- take()
- count()
- бродкаст-переменные

In [ ]:
print(rdd.collect())

In [ ]:
broadcast_var = spark.sparkContext.broadcast([10, 20, 30, 40, 50])

In [ ]:
rdd_with_broadcast = rdd.map(lambda x: x + broadcast_var.value[0])

In [ ]:
rdd_with_broadcast.collect()

In [ ]:
from random import randint

In [ ]:
data_ = [randint(1, 15) for _ in range(10)]

In [ ]:
data_

In [ ]:
rdd_ = spark.sparkContext.parallelize(data_)

In [ ]:
rdd_.collect()

In [ ]:
rdd_.repartition(5).collect() # шафл

In [ ]:
rdd_.coalesce(2).collect() # без шафла

In [ ]:
rdd_.count()

In [ ]:
rdd_.cache()

In [ ]:
rdd_.persist()

In [ ]:
rdd_.unpersist()

In [ ]:
a = rdd_.persist()

In [ ]:
a.count()

In [ ]:
rdd_.count()

### Объединение rdd

In [ ]:
rdd1 = spark.sparkContext.parallelize([1, 2, 3]).repartition(1)
rdd2 = spark.sparkContext.parallelize([4, 5, 6]).repartition(1)

In [ ]:
union_rdd = rdd1.union(rdd2)

In [ ]:
union_rdd.collect()

In [ ]:
rdd_pairs1 = spark.sparkContext.parallelize([(1, "A"), (2, "B"), (3, "C")])
rdd_pairs2 = spark.sparkContext.parallelize([(1, "X"), (2, "Y"), (4, "Z")])

In [ ]:
rdd_pairs1

In [ ]:
rdd_pairs1.collect()

In [ ]:
joined_rdd = rdd_pairs1.join(rdd_pairs2)

In [ ]:
joined_rdd.collect()

In [ ]:
from pyspark.sql.types import *

In [ ]:
rdd_next = spark.sparkContext.parallelize([(1, "Alice"), (2, "Bob"), (3, "Charlie")])

In [ ]:
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True)
])

In [ ]:
df = spark.createDataFrame(rdd_next, schema)
df.show()

In [ ]:
df.collect()

### Работа с JSON

In [ ]:
json_rdd = spark.sparkContext.parallelize([
    '{"id": 1, "name": "Alice"}',
    '{"id": 2, "name": "Bob"}'
])

In [ ]:
df_j = spark.read.json(json_rdd)
df_j.show()

In [ ]:
df_j.collect()

### WC

In [ ]:
text_rdd = spark.sparkContext.parallelize([
    "Spark is awesome",
    "Spark is fast",
    "Hello Spark"
])

In [ ]:
word_counts = (text_rdd
               .flatMap(lambda line: line.split(" ")) 
               .map(lambda word: (word, 1))           
               .reduceByKey(lambda a, b: a + b))

In [ ]:
word_counts.collect()

### Вычисление среднего

In [ ]:
salary_data = [("IT", 100), ("HR", 80), ("IT", 120), ("HR", 90)]
rdd = spark.sparkContext.parallelize(salary_data)

In [ ]:
result = (rdd
          .mapValues(lambda x: (x, 1))               
          .reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1]))  
          .mapValues(lambda x: x[0]/x[1]))            

In [ ]:
result = (rdd
          .mapValues(lambda x: (x, 1)))

In [ ]:
result.collect()

In [ ]:
result_ = result.reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1]))

In [ ]:
result_.collect()

In [ ]:
result__ = result_.mapValues(lambda x: x[0]/x[1])

In [ ]:
result__.collect()

### flatMap

разбиение строк на слова 

In [ ]:
lines = ["Hello world", "Apache Spark", "PySpark RDD"]
rdd = spark.sparkContext.parallelize(lines)

In [ ]:
words_rdd = rdd.flatMap(lambda line: line.split(" "))

In [ ]:
print(words_rdd.collect())

### Развертывание вложенных структур

In [ ]:
data = [("A", [1, 2, 3]), ("B", [4, 5])]
rdd = spark.sparkContext.parallelize(data)

In [ ]:
flattened = rdd.flatMap(lambda x: [(x[0], num) for num in x[1]])

In [ ]:
print(flattened.collect())

### Оптимизация

In [ ]:
data = [("A", 10), ("B", 20), ("A", 30), ("B", 40)]
rdd = spark.sparkContext.parallelize(data)

In [ ]:
rdd.collect()

In [ ]:
result = rdd.aggregateByKey(
    (0, 0),                         # Нулевое значение
    lambda acc, val: (acc[0] + val, acc[1] + 1),  # Локальная агрегация
    lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1]))

In [ ]:
result.collect()

In [ ]:
result = rdd.combineByKey(
    lambda value: (value, 1, value),                  # Создаем аккумулятор
    lambda acc, value: (acc[0] + value, acc[1] + 1, max(acc[2], value)),
    lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1], max(acc1[2], acc2[2])))

print(result.collect())